In [86]:
!pwd

/opt/ml/input/fighting/Test


In [1]:
import os
import sys
sys.path.append('..')
from Utils import model_recommend_movies
from interaction_model import Inference
import pandas as pd
from tqdm import tqdm

In [2]:
ratings = pd.read_csv('/opt/ml/input/fighting/CSV/ml-25m/ratings.csv')

In [3]:
ratings

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434


In [4]:
ratings['userId'].nunique(), ratings['movieId'].nunique()

(162541, 59047)

In [5]:
data_path = "/opt/ml/input/fighting/CSV/"
inter_mbti_ml_title_movieid = pd.read_csv(data_path+"inter_mbti_ml_title_movieId.csv")

In [6]:
inter_mbti_ml_movieid = list(inter_mbti_ml_title_movieid['movieId'].values)

In [7]:
# mbti와 movielens에 공통으로 있는 영화에 대한 interaction만 고려
ratings = ratings[ratings['movieId'].isin(inter_mbti_ml_movieid)]

In [8]:
ratings['userId'].nunique(), ratings['movieId'].nunique()

(162537, 3213)

## 9:1로 train, test 유저나누기

In [9]:
import random

user_list = list(ratings['userId'].unique())
test_size = int(len(user_list) * 0.1)
random.seed(2023)
test_user = random.sample(user_list, test_size)
train_user = list(set(user_list)-set(test_user))

In [10]:
ratings['userId'].nunique(), ratings['movieId'].nunique()

(162537, 3213)

In [11]:
train_df = ratings[ratings['userId'].isin(train_user)]

In [12]:
train_df['userId'].nunique(), train_df['movieId'].nunique()

(146284, 3202)

In [13]:
train_df.to_csv("train_df.csv", index=False)

In [14]:
test_df = ratings[ratings['userId'].isin(test_user)]

In [15]:
test_df['userId'].nunique(), test_df['movieId'].nunique()

(16253, 3005)

## train_df으로 학습한 annoy 함수 만들기

In [16]:
!pwd

/opt/ml/input/fighting/interaction_model


In [ ]:
from scipy import sparse
from scipy.sparse import csr_matrix, lil_matrix
import implicit
import annoy

data_path = "/opt/ml/input/fighting/CSV/"
ratings = pd.read_csv("./train_df.csv")
# movies = pd.read_csv(data_path + "ml-25m/movies.csv")

# ratings = ratings.merge(movies[["movieId","title"]],how="left",on = "movieId")
ratings['rating'] = 1
ratings.drop(columns = "timestamp",inplace = True)
inter_mbti_ml_title_movieid = pd.read_csv(data_path+"inter_mbti_ml_title_movieId.csv")

user2idx = {k:v for v,k in enumerate(ratings['userId'].unique())}
item2idx = {k:v for v,k in enumerate(ratings['movieId'].unique())}
idx2item = {v:k for v,k in enumerate(ratings['movieId'].unique())}

ratings['useridx'] = ratings['userId'].map(user2idx) # userId -> 인덱스로 바꿔줍니다.
ratings['movieidx'] = ratings['movieId'].map(item2idx) # movieId -> 인덱스로 바꿔줍니다.

sparse_user_item = sparse.csr_matrix((ratings['rating'].astype(float),
                                      (ratings['useridx'], ratings['movieidx']))) # csr_matrix 생성

als_model = implicit.als.AlternatingLeastSquares(
                                factors = 64,
                                regularization = 0.1,
                                iterations = 200,
                                calculate_training_loss = False,
                                use_gpu = True
                            ) # 모델 init

als_model.fit(sparse_user_item) # 학습
item_vecs = als_model.item_factors.to_numpy() # item vector만 빼옵니다.

In [18]:
# inter_mbti_ml_title_movieid도 인덱싱처리
inter_mbti_ml_title_movieid['movieidx'] = inter_mbti_ml_title_movieid['movieId'].map(item2idx)
inter_mbti_ml_title_movieid.dropna(inplace = True)
inter_mbti_ml_title_movieid['movieidx'] = inter_mbti_ml_title_movieid['movieidx'].astype(int)

# 유사도 측정방법
# angular : cosine similarity
# euclidean : l2-norm distance
# manhattan : l1-norm distance
# hamming : binary vector에
similar_items_index = annoy.AnnoyIndex(64,'angular')
n_trees = 40
for idx in inter_mbti_ml_title_movieid['movieidx']:
    similar_items_index.add_item(idx2item[idx],item_vecs[idx]) # idx -> movieId로 바꿔서 annoy에 저장합니다.
similar_items_index.build(n_trees)
similar_items_index.save("ALS_angular_64")

True

In [66]:
item_vecs.shape

(3202, 64)

## test 유저별 interaction 영화 id list

In [19]:
grouped = test_df.groupby('userId')
test_user_movie = grouped['movieId'].apply(list)

In [20]:
test_user_movie

userId
10        [1, 32, 50, 110, 150, 193, 296, 318, 356, 380,...
18        [1, 7, 17, 50, 110, 141, 173, 253, 266, 288, 2...
41        [6, 44, 70, 110, 145, 163, 168, 170, 198, 353,...
50        [1, 32, 150, 208, 296, 344, 356, 377, 455, 480...
54        [7, 337, 342, 593, 1101, 1219, 1246, 2028, 230...
                                ...                        
162494    [14, 32, 562, 924, 1103, 1367, 1956, 2160, 257...
162500    [32, 50, 293, 296, 318, 608, 1089, 1213, 1704,...
162511    [1088, 1197, 1290, 2145, 2161, 2193, 2248, 310...
162513    [16, 50, 104, 110, 150, 173, 235, 266, 296, 31...
162539    [110, 161, 356, 480, 541, 608, 780, 912, 920, ...
Name: movieId, Length: 16253, dtype: object

In [21]:
test_user_movie_dict = dict(test_user_movie)

In [67]:
test_user_movie_dict

{10: [1,
  32,
  50,
  110,
  150,
  193,
  296,
  318,
  356,
  380,
  457,
  480,
  588,
  593,
  608,
  780,
  924,
  1186,
  1347,
  1962,
  2094,
  2505,
  2572,
  2915,
  3361,
  3863,
  4361,
  5810,
  6502,
  7444,
  53953],
 18: [1,
  7,
  17,
  50,
  110,
  141,
  173,
  253,
  266,
  288,
  293,
  296,
  337,
  344,
  353,
  356,
  357,
  380,
  457,
  480,
  500,
  539,
  541,
  586,
  587,
  588,
  593,
  595,
  597,
  608,
  648,
  733,
  736,
  778,
  780,
  784,
  924,
  1036,
  1080,
  1097,
  1101,
  1127,
  1136,
  1197,
  1208,
  1214,
  1220,
  1222,
  1225,
  1265,
  1270,
  1278,
  1287,
  1304,
  1307,
  1517,
  1527,
  1573,
  1580,
  1584,
  1610,
  1653,
  1682,
  1704,
  1721,
  1917,
  1923,
  1968,
  2000,
  2006,
  2028,
  2167,
  2174,
  2268,
  2291,
  2529,
  2706,
  2710,
  2712,
  2716,
  2791,
  2797,
  2858,
  2916,
  2918,
  3101,
  3105,
  3271,
  3355,
  3421,
  3471,
  3499,
  3578,
  3897,
  3996,
  4085,
  4262,
  4306,
  4720,
  5064,
  5445

## test_유저의 valid score 확인(한명에 대한 실험)

### 5:5로 test_유저별 영화기록을 입력, valid 나누기

In [22]:
valid_ratio = 0.5

In [23]:
sys.path.append(os.path.join(os.path.abspath(os.path.curdir),".."))
from Utils.utils import model_recommend_movies

In [24]:
# 모델명
model_name = './ALS_angular_64'

In [35]:
list(test_user_movie_dict.keys())[0]

10

In [36]:
# inference 입력 영화 id
userId = list(test_user_movie_dict.keys())[0]
test_user_prefer_movieids = random.sample(test_user_movie_dict[userId], int(len(test_user_movie_dict[userId])*(1-valid_ratio)))

In [37]:
# valid 영화 id
test_user_valid_movieids = list(set(test_user_movie_dict[userId]) - set(test_user_prefer_movieids))

### 추천 영화와 valid 영화 교집합 비율 확인

In [38]:
# 추천받은 영화 id
recommend_movie_ids = model_recommend_movies("INFP","5w4", test_user_prefer_movieids, 10, model_name)


In [39]:
# 추천받은 영화와 valid 영화 교집합
len(set(recommend_movie_ids) & set(test_user_valid_movieids))

3

In [40]:
# valid 영화 집합
len(test_user_valid_movieids)

16

In [41]:
# 유저 3에 대한 valid score
len(set(recommend_movie_ids) & set(test_user_valid_movieids)) / len(test_user_valid_movieids)

0.1875

## test_유저의 valid score 확인(일반화)

In [42]:
sys.path.append(os.path.join(os.path.abspath(os.path.curdir),".."))
from Utils.utils import model_recommend_movies
# 모델명
model_name = './ALS_angular_64'

In [43]:
scores = []
for user, movie_list in tqdm(test_user_movie_dict.items()):
    movie_list_len = len(movie_list)
    user_prefer_movieids = random.sample(movie_list, movie_list_len//2)     # inference 입력 영화 id list
    user_valid_movieids = list(set(movie_list) - set(user_prefer_movieids)) # valid 영화 id list
    user_rec_movieids = model_recommend_movies("INFP","5w4",user_prefer_movieids,10,model_name) #추천 영화 id list
    inter_cnt = len(set(user_valid_movieids) & set(user_rec_movieids))      # valid 영화와 추천 영화 교집합 개수
    score = inter_cnt / len(user_valid_movieids)                            # valid 영화 개수
    scores.append(score)

100%|██████████| 16253/16253 [03:27<00:00, 78.45it/s] 


In [44]:
# scores

In [45]:
import numpy as np
mean = np.mean(scores)
print(f'test_유저의 valid 스코어의 평균: {mean}')
stdev = np.std(scores)
print(f'test_유저의 valid 스코어의 표준편차: {stdev}')

test_유저의 valid 스코어의 평균: 0.43696534590641645
test_유저의 valid 스코어의 표준편차: 0.2213002550253481
